In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
from __future__ import print_function
from numpy import *
from scipy import *

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import reload
import matplotlib.patches as patches

import netCDF4 as nc
import numpy as np
import scipy as sp

import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

# matplotlib.use('Agg')
# plt.style.use('seaborn-whitegrid')
import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
import mocsy
import CCCma
import CCCma_stations as cs

from scipy import optimize
import arrow
import gsw
import pandas as pd
import xarray as xr
%matplotlib inline

/home/tjarniko/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
def preind_dic_ncmaker(startind, endind, year):
#1 open given boundary conditions file and findpco2 and potential density 
    daymon = [31,28,31,30,31,30,31,31,30,31,30,31]
    daymon_LY = [31,29,31,30,31,30,31,31,30,31,30,31]

    year_ar = []
    noday = 365
    if year == 2016:
        t_daymon = daymon_LY
        noday = 366
    else:
        t_daymon = daymon

    for m in range(1,13):
        if m>=10:
            tm = str(m)
        if m<10:
            tm = '0' + str(m)
        print(tm)
        for d in range(1,t_daymon[m-1]+1):
            if d>=10:
                td = str(d)
            if d<10:
                td = '0' + str(d)

            tstr = 'y' + str(year) + 'm' + tm + 'd' + td
            year_ar.append(tstr)
                            
    for ind in range(startind,endind):

        print(year_ar[ind])
        test_LO = nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_' + year_ar[ind] +'.nc')        
        dsin = test_LO
        #output file
        ncname = './preind_DIC/PREIND_BC_v201905_' + year_ar[ind] +'.nc'
        print(ncname)
        dsout = nc.Dataset(ncname, "w", format="NETCDF4")
        #Copy dimensions
        for dname, the_dim in dsin.dimensions.items():
            print(dname)
            print(len(the_dim))
            dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
        # Copy variables
        for v_name, varin in dsin.variables.items():
            print(v_name)
            if (v_name != 'DIC'):
                outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
            # Copy variable attributes
                outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

                outVar[:] = varin[:]
            if (v_name == 'DIC'):
                print('not copying DIC!')
        w = nc.Dataset('./preind_DIC/diseq/LO_AOUmethod_stoicCO_diseq_'+ year_ar[ind] +'_preind_DIC.nc')
        piDIC = w['preindustrial_DIC']['preind_dic']
        t_DIC = np.zeros([1, 40, 1, 950])
        t_DIC[0,:,0,:] = piDIC[:,:]
        
        DIC = dsout.createVariable('DIC',np.float32,('time_counter','deptht','yb','xbT'))
        DIC[:] = t_DIC
        
        dsout.close()

In [5]:
# startind = 0
# endind = 365
# year = 2015
# preind_dic_ncmaker(startind, endind, year)

In [6]:
# w = nc.Dataset('./PREIND_BC_v201905_AndrewAge_y2015m01d01.nc')

In [15]:
## code for JS

test_LO = nc.Dataset('/data/tjarniko/MEOPAR/tracers/north/Dosser_north_TEOS10_DICTA_preind_080819.nc')        
dsin = test_LO
#output file
ncname = './preind_DIC/PREIND_BC_v201905_JS.nc'
print(ncname)
dsout = nc.Dataset(ncname, "w", format="NETCDF4")
#Copy dimensions
for dname, the_dim in dsin.dimensions.items():
    print(dname)
    print(len(the_dim))
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# Copy variables
for v_name, varin in dsin.variables.items():
    print(v_name)
    if (v_name != 'DIC'):
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
    # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

        outVar[:] = varin[:]
    if (v_name == 'DIC'):
        print('not copying DIC!')
w = nc.Dataset('/data/tjarniko/MEOPAR/analysis-tereza/notebooks/carbon_dev/PI_BOUND_COND/CLEAN/preind_DIC/JS_diseq_preind_DIC.nc')
piDIC = w['preindustrial_DIC']['preind_dic']
#     t_DIC = np.zeros([1, 40, 1, 950])
#     t_DIC[0,:,0,:] = piDIC[:,:]

DIC = dsout.createVariable('DIC',np.float32,('time_counter','deptht','yb','xbT'))
DIC[:] = piDIC

dsout.close()

./preind_DIC/PREIND_BC_v201905_JS.nc
time_counter
12
deptht
40
yb
10
xbT
30
DIC
not copying DIC!
TA
deptht
time_counter
vosaline
votemper


In [9]:
w = nc.Dataset('/data/tjarniko/MEOPAR/analysis-tereza/notebooks/carbon_dev/PI_BOUND_COND/CLEAN/preind_DIC/JS_diseq_preind_DIC.nc')

In [13]:
print(w['preindustrial_DIC']['preind_dic'])

<class 'netCDF4._netCDF4.Variable'>
float32 preind_dic(month, depth, xval, yval)
path = /preindustrial_DIC
unlimited dimensions: 
current shape = (12, 40, 10, 30)
filling on, default _FillValue of 9.969209968386869e+36 used

